In [817]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [818]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import regex as re
driver = webdriver.Firefox()



In [819]:

#driver.get("http://selenium.dev")


In [820]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [821]:
#driver.get('http://www.yahoo.com')
#assert 'Yahoo' in driver.title



In [822]:
#elem = driver.find_element(By.NAME, 'p')  # Find the search box
#elem.send_keys('seleniumhq' + Keys.RETURN)

In [823]:
#driver.quit()

In [824]:
driver.get('https://www.ncbi.nlm.nih.gov/sra')


In [825]:
# ngs_seq_search = input("Type the name of technology and organism you want to find:")
elem = driver.find_element(By.NAME, 'term')
elem.send_keys('direct RNA-seq and bacteria and ONT' + Keys.RETURN)
# elem.send_keys(ngs_seq_search + Keys.RETURN)



In [826]:
src = driver.page_source
# BeautifulSoup(src.content, "html.parser")
parser = BeautifulSoup(src, "lxml")
table = parser.find("table", attrs={"class": "rprtid"})
dl_data = parser.find_all("dd")
# print(dl_data)

for dlitem in dl_data:
    accession_list = dlitem.string
    print(accession_list)


   
    # print(accession_list)
    # df1 = pd.DataFrame(accession_list)
    # print(df1)
# print(accession_list)
    # accession_list_df = dl_data

    # import numpy as np
    # array = np.array([1.2, accession_list_df], dtype=object)
    # print(array)
# print(accession_list_df)
    # array.to_csv('/Users/irazemagarcia/MS_Bioinformatics/Advanced Data Management/Final project/Final_script/data/accession_list.csv')
    # print(accession_list)
    # dlitem.string
    # print (dlitem.string)

    # df = DataFrame(eval(data,columns=columnsss ))

    # df = accession_list
    # print(df)
    # df.to_csv('/Users/irazemagarcia/MS_Bioinformatics/Advanced Data Management/Final project/Final_script/data/file.csv')

# header = [dt.getText() for dt in table.findAll('dd', limit=2)[0].findAll('dt')]
# header = table.findAll('dl')
# print(parser)
# print(src)
# print(table)
# print(header)
# https://stackoverflow.com/questions/32475700/using-beautifulsoup-to-extract-specific-dl-and-dd-list-elements

In [827]:
#Make a list
data = []
for i in range(0, len(dl_data)):
    rows = dl_data[i].get_text()
    data.append(rows)

In [828]:
#Clean list
data = data[:-1]
print(data)
len(data)

[]


0

In [829]:
#Get accession numbers and turn them into a text file. Important for METADATA
textfile = open("accn.txt", "w")
for an in data:
    textfile.write(an + "\n")
textfile.close()

In [831]:
#Go to run selector
selector = driver.find_element(By.ID, 'id-sra-run-selector-popup')
selector.click()

In [ ]:
#Get run selector table
src = driver.page_source
parser = BeautifulSoup(src, "lxml")
selector_table = parser.find("table", attrs={"class": "ui selectable striped celled very compact table single line"})
#selector_table = parser.find("div", attrs={"id": "ph-rs-table"})
headers = selector_table.find_all("th")
headerlist = [h.text.strip() for h in headers [1:]] #Delete spaces and other format

# print(selector_table)

In [ ]:
# headerlist

In [ ]:
rows = selector_table.findAll('tr')[1:]

In [ ]:
# rows

In [ ]:
#Get rows
sequences_data = [[td.getText().strip() for td in rows[i].findAll('td')[1:]] for i in range(len(rows))]

In [ ]:
# sequences_data
# len(rows)

In [ ]:
sequencesdf = pd.DataFrame(sequences_data, columns=headerlist)

In [ ]:
# sequencesdf

,1Run,2BioProject,3BioSample,4Center Name,5DATASTORE filetype,6Experiment,7Isolation_Source,8Library Name,9LibrarySelection,10LibrarySource,...,22Bases,23Bytes,24create_date,25version,26BIOMATERIAL_PROVIDER,27AssemblyName,28type-material,29HOST,30Isolate,31primary_investigator
0,SRR11550236,PRJNA497103,SAMN10247560,UNIVERSITY OF ARKANSAS FOR MEDICAL SCIENCES,"bam, run.zq, sra",SRX8120039,Cultivation,EcoliTypeStrain_dcDNA,RANDOM,TRANSCRIPTOMIC,...,176.75 M,143.86 Mb,2020-04-1720:26:00Z,1,,assembly,neotype strain of Escherichia coli,,,
1,SRR11550247,PRJNA497103,SAMN10247560,UNIVERSITY OF ARKANSAS FOR MEDICAL SCIENCES,"bam, run.zq, sra",SRX8120028,Cultivation,EcoliTypeStrain_dRNA,RANDOM,TRANSCRIPTOMIC,...,237.95 M,184.93 Mb,2020-04-1720:26:00Z,1,,assembly,neotype strain of Escherichia coli,,,
2,SRR13570801,PRJNA575545,SAMN12900250,CENTRE FOR GENOMIC REGULATION,"nanopore, run.zq, sra",SRX9968998,NA,UNM_short,Oligo-dT,SYNTHETIC,...,15.87 M,13.55 Mb,2021-01-2809:22:00Z,1,,,,NA,NA,
3,SRR18061002,PRJNA805453,SAMN25861597,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213007,laboratory,RDMIN20210517,PolyA,TRANSCRIPTOMIC,...,394.95 M,357.82 Mb,2022-02-2405:57:00Z,1,Dr. David Rasko,,,,,
4,SRR18061003,PRJNA805453,SAMN25861595,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213006,laboratory,RDMIN20210507,PolyA,TRANSCRIPTOMIC,...,1.25 G,1.08 Gb,2022-02-2405:48:00Z,1,Dr. David Rasko,,,,,
5,SRR18061004,PRJNA805453,SAMN25861596,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213005,laboratory,RDMIN20201211,PolyA,TRANSCRIPTOMIC,...,2.02 G,1.76 Gb,2022-02-2405:52:00Z,1,Dr. David Rasko,,,,,
6,SRR18061005,PRJNA805453,SAMN25861598,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213004,laboratory,RDMIN20200702,PolyA,TRANSCRIPTOMIC,...,2.21 G,1.91 Gb,2022-02-2405:46:00Z,1,Dr. David Rasko,,,,,
7,SRR18070402,PRJNA805453,SAMN25861597,"UNIVERSITY OF MARYLAND, BALTIMORE",nanopore,SRX14222110,laboratory,RDMIN20210517-fast5,PolyA,TRANSCRIPTOMIC,...,,,,,Dr. David Rasko,,,,,
8,SRR18070403,PRJNA805453,SAMN25861595,"UNIVERSITY OF MARYLAND, BALTIMORE",nanopore,SRX14222109,laboratory,RDMIN20210507-fast5,PolyA,TRANSCRIPTOMIC,...,,,,,Dr. David Rasko,,,,,
9,SRR18070404,PRJNA805453,SAMN25861596,"UNIVERSITY OF MARYLAND, BALTIMORE",nanopore,SRX14222108,laboratory,RDMIN20201211-fast5,PolyA,TRANSCRIPTOMIC,...,,,,,Dr. David Rasko,,,,,


In [ ]:
#Pull all run IDs
runid = sequencesdf['1Run'].to_list()
print(runid)

['SRR11550236', 'SRR11550247', 'SRR13570801', 'SRR18061002', 'SRR18061003', 'SRR18061004', 'SRR18061005', 'SRR18070402', 'SRR18070403', 'SRR18070404', 'SRR18070405', 'SRR21654270', 'SRR21654271', 'SRR21654272', 'SRR23886068']


In [ ]:
type(sequencesdf)

pandas.core.frame.DataFrame

In [ ]:
#Pull only run experiments with fast5 files
fast5_df = sequencesdf.loc[sequencesdf['5DATASTORE filetype']=='nanopore']
print(fast5_df)


           1Run  2BioProject    3BioSample  \
7   SRR18070402  PRJNA805453  SAMN25861597   
8   SRR18070403  PRJNA805453  SAMN25861595   
9   SRR18070404  PRJNA805453  SAMN25861596   
10  SRR18070405  PRJNA805453  SAMN25861598   
12  SRR21654271  PRJNA882509  SAMN30941647   

                                         4Center Name 5DATASTORE filetype  \
7                   UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
8                   UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
9                   UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
10                  UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
12  SIR RUN RUN SHAW HOSPITAL, ZHEJIANG UNIVERSITY...            nanopore   

    6Experiment        7Isolation_Source        8Library Name  \
7   SRX14222110               laboratory  RDMIN20210517-fast5   
8   SRX14222109               laboratory  RDMIN20210507-fast5   
9   SRX14222108               laboratory  RDMIN20201211-fast5   


,1Run
7,SRR18070402
8,SRR18070403
9,SRR18070404
10,SRR18070405
12,SRR21654271


In [ ]:
df5 = fast5_df['1Run'].to_list()
print(df5)

['SRR18070402', 'SRR18070403', 'SRR18070404', 'SRR18070405', 'SRR21654271']


In [ ]:
#Fast5 accession number
file = open("an_f5.txt", "w")
for an in df5:
    file.write(an + "\n")
file.close()

In [ ]:
#Gather all links to data-access
links =[]
for run in runid:
    sra_data_access = f'https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc={run}&display=data-access'
    separate = re.findall(r'(https?://\S+)', sra_data_access)
    links.append(separate)

In [ ]:
print(links)

[['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR11550236&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR11550247&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR13570801&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061002&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061003&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061004&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061005&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18070402&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18070403&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18070404&display=data-access'], ['https://trace.ncbi.nlm.nih.

In [ ]:
#Get only fast5 files
fast5 = []
for run in df5:
    sra_data_access = f'https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc={run}&display=data-access'
    separate = re.findall(r'(https?://\S+)', sra_data_access)
    datalink = fast5.append(separate)

    

In [ ]:
# my_list = (iter(fast5))
df1 = pd.DataFrame(fast5)
# print(df1)
#List of links
# fast5[0]
# fast5[1]

# fast5[3]
# fast5[4]
# my_list = next(iter(fast5))
# print(my_list)
# df1.columns = range(df.shape[1])
print(df1)
df1.to_csv('/Users/irazemagarcia/MS_Bioinformatics/Advanced Data Management/Final project/Final_script/data/fast5url.csv',header=None,index=False)


                                                   0
0  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
1  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
2  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
3  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
4  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...


In [ ]:
import bs4 as bs



df_link = pd.read_csv('/Users/irazemagarcia/MS_Bioinformatics/Advanced Data Management/Final project/Final_script/data/fast5url.csv') 
# df_link.columns = ['Col']
print(df_links.head)

# for link in df_link.iterrows():
#     url = link[1]['Col']
#     x = urllib2.urlopen(url)
#     new = x.read()
#     # Code does not even get past here as far as I checked
#     soup = bs.BeautifulSoup(new,"lxml")
#     for text in soup.find_all('a',href = True):
#         text.append((text.get('href')))

<bound method NDFrame.head of                                                  Col
0  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
1  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
2  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
3  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...>


In [ ]:
#Df of list of links
links_df = pd.DataFrame(links, columns=['Data_access_links'], dtype= str)
links_df

,Data_access_links
0,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
1,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
2,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
3,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
4,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
5,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
6,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
7,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
8,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
9,https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...


In [ ]:
#Text file for bash command
textfile_da = open("data_access.txt", "w")
for link in links:
    textfile_da.write(link + "\n")
textfile_da.close()

TypeError: can only concatenate list (not "str") to list

In [ ]:
with open('links.txt','w') as tfile:
    tfile.write('\n'.join(links))

TypeError: sequence item 0: expected str instance, list found

In [ ]:
#!/bin/bash



In [ ]:
def data_acc():
    for run in runid:
        sra_data_access = f'https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc={run}&display=data-access'
        links = re.findall(r'(https?://\S+)', sra_data_access)
    # print(sra_data_access)
        return str(links)
        print(links)
    
# da_links = []
# for i in range(0,len(links)):
#     da_links.append(links)
#     print(da_links)




In [ ]:
str(data_acc)

'<function data_acc at 0x7fe028ee6940>'

In [ ]:
# re.search("(?P<url>https?://[^\s]+)", sra_data_access).group("url")

print(re.findall(r'(https?://\S+)', sra_data_access))

# for line in sra_data_access:
#     line = line.strip()
#     # items = line.split()
#     print(line)
#     # print(items)
#     word = line[1]


['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR23886068&display=data-access']


In [ ]:
# link_regex = re.compile("((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)", re.DOTALL)


In [ ]:
# #List
# links1 = re.findall(link_regex, sra_data_access)
# print(links1[0])
# # links

('https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR23886068&display=data-access', 'https', '//', '//', '', 's', '')


In [ ]:
# #Conver string to list
def Convert(string):
    links = list(string.split(" "))
    return links

# # Code
print(Convert(sra_data_access))
# seq_list = list(sra_data_access.strip("*"))
# print(seq_list)

['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR23886068&display=data-access']


In [ ]:
# #Get each href link to pull titles and append with accession list
# for accessions in data:
#     href = f'href="sra/{accessions}[accn]"'
#     print(href)

href="sra/SRX19697903[accn]"
href="sra/SRX17654229[accn]"
href="sra/SRX17654228[accn]"
href="sra/SRX17654227[accn]"
href="sra/SRX14222110[accn]"
href="sra/SRX14222109[accn]"
href="sra/SRX14222108[accn]"
href="sra/SRX14222107[accn]"
href="sra/SRX14213007[accn]"
href="sra/SRX14213006[accn]"
href="sra/SRX14213005[accn]"
href="sra/SRX14213004[accn]"
href="sra/SRX9968998[accn]"
href="sra/SRX8120039[accn]"
href="sra/SRX8120028[accn]"


In [ ]:
parser_title = BeautifulSoup(src, "lxml")
table_title = parser.find("div", attrs={"class": "rslt"})
print(table_title)
# title_data = parser_title.find_all("/p")
# print(title_data)


<div class="rslt"><p class="title"><a href="/sra/SRX19697903[accn]" ref="ordinalpos=1&amp;ncbi_uid=27005456&amp;link_uid=27005456"><b>ONT</b> <b>Direct</b> RNA Sequencing of E. coli K12 MG1655</a></p><div class="supp"><p class="details expand e-hidden">1 OXFORD_NANOPORE (MinION) run: 392,041 spots, 301.3M bases, 274.2Mb downloads</p></div><div class="aux"><div class="resc"><dl class="rprtid"><dt>Accession: </dt> <dd>SRX19697903</dd></dl></div><p class="links nohighlight"></p></div></div>
[]


In [ ]:
#Df for accession numbers 
df = pd.DataFrame(data, columns=['Accession_number'], dtype= str)


In [ ]:
#Convert df into csv files for the accession numbers
df.to_csv('/Users/irazemagarcia/MS_Bioinformatics/Advanced Data Management/Final project/Final_script/data/accession_list1.csv')

In [ ]:
#Find all results
# b = []
# xpath = '/html/body/div[1]/div[1]/form/div[1]/div[4]/div/div[5]/div[{}]/div[2]/p/a'

# xpath = while driver.find_elements(By.XPATH, '/html/body/div[1]/div[1]/form/div[1]/div[4]/div/div[5]/div[1]/div[2]/p/a')



In [ ]:
#
# for i in accn:

# title = driver.find_element(By.LINK_TEXT, 'href="/sra/{})